In [1]:
from torchrecsys.models import NCF
from torchrecsys.external_datasets import Movielens_1M
from torchrecsys.datasets import InteractionsDataset
from pytorch_lightning import Trainer
import pandas as pd
import torch
from torch.utils.data import DataLoader

In [2]:
data = Movielens_1M()
ratings, users, movies = data.load()

In [3]:
#Preprocess users
users['gender'], uniques = pd.factorize(users['gender'])
users['occupation'], uniques = pd.factorize(users['occupation'])
users['zip'], uniques = pd.factorize(users['zip'])
##Set category dtype
users['gender'] = users.gender.astype('category')
users['occupation'] = users.occupation.astype('category')
users['zip'] = users.zip.astype('category')

#Preprocess movies, 
##categories to index
movies['title'], uniques = pd.factorize(movies['title'])
movies['genres'], uniques = pd.factorize(movies['genres'])
##Set category dtype
movies['title'] = movies.title.astype('category')
movies['genres'] = movies.genres.astype('category')

In [4]:
dataset = InteractionsDataset(ratings, users, movies, item_id="movie_id", interaction_id="rating")
train = DataLoader(dataset, batch_size=512)

In [5]:
model = NCF(dataset.data_schema)

In [6]:
trainer = Trainer(max_epochs=1)
trainer.fit(model, train)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/jaimeferrando/miniconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:118: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name           | Type       | Params
----------------------------------------------
0 | user_features  | ModuleList | 27.7 K
1 | item_features  | ModuleList | 33.5 K
2 | user_embedding | Embedding  | 386 K 
3 | item_embedding | Embedding  | 288 K 
4 | mlp            | Sequential | 222 K 
5 | final_linear   | Linear     | 346   
6 | criterion      | MSELoss    | 0     
----------------------------------------------
959 K     Trainable params
0         Non-trainable params
959 K     Total params
3.839     Total estimated model params size (MB)
/Users/jaimeferrando/mi

Training: 0it [00:00, ?it/s]

In [7]:
pair = torch.tensor([[1,1193]])
context = torch.tensor([])
user = torch.tensor([[0,1,0,1]])
item = torch.tensor([[0,0]])

model(pair, context, user, item)

tensor([[4.5262]], grad_fn=<AddmmBackward0>)

In [25]:
import numpy as np

interactions = np.array(
        [
            [1, 1193, 5],
            [1, 661, 3],
            [1, 914, 3],
            [2, 3408, 4],
            [2, 2355, 5],
            [2, 3408, 4],
            [2, 2355, 5],
        ],
        dtype=int,
    )
interactions = pd.DataFrame(interactions, columns=["user_id", "item_id", "rating"],)
users = np.array(
        [
            [1, "F", 1, "10", "48067"],
            [2, "M", 56, "16", "70072"],
            [3, "M", 25, "15", "55117"],
            [4, "M", 45, "7", "02460"],
            [5, "M", 25, "20", "55455"],
        ]
    )
users = pd.DataFrame(
    users, columns=["user_id", "sex", "age", "occupation", "postal_code"]
)

items = np.array(
        [
            [1, "Toy Story (1995)", "Animation|Children's|Comedy"],
            [2, "Jumanji (1995)", "Adventure|Children's|Fantasy"],
            [3, "Grumpier Old Men (1995)", "Comedy|Romance"],
            [4, "Waiting to Exhale (1995)", "Comedy|Drama"],
            [5, "Father of the Bride Part II (1995)", "Comedy"],
        ]
    )
items = pd.DataFrame(items, columns=["item_id", "title", "genre"])

In [20]:
a = InteractionsDataset(
        interactions,
        users,
        items,
        interaction_id="rating",
    )

In [26]:
users.dtypes

user_id        object
sex            object
age            object
occupation     object
postal_code    object
dtype: object